In [1]:
# !pip install ipywidgets  # for vscode
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import json

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from transformers import AutoTokenizer, AutoModel
from transformers import ElectraModel, ElectraTokenizer

from sklearn.utils.class_weight import compute_class_weight

In [3]:
#KoBERT
# tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
# model = AutoModel.from_pretrained("monologg/kobert")

# KoELECTRA-Base
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
model = ElectraModel.from_pretrained("monologg/koelectra-base-discriminator")

Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
class sentencEmojiDataset(Dataset):
    def __init__(self, directory, tokenizer):
        
        data = pd.read_csv(directory, encoding='UTF-8')
        
        self.tokenizer = tokenizer
        self.sentences = list(data.iloc[:,0])
       
        emojis = list(data.iloc[:,1])
        emojis_unique = list(set(emojis))
        
        self.labels = [emojis_unique.index(i) for i in emojis]
 
        self.labels_dict = {'key': range(len(emojis_unique)), 'value': emojis_unique}
        
    def __getitem__(self, i): #collate 이전 미리 tokenize를 시켜주자
        tokenized = self.tokenizer(str(self.sentences[i]), return_tensors='pt')
        
        #아래 세 개는 tokenizer가 기본적으로 반환하는 정보. BERT의 input이기도 함
        input_ids = tokenized['input_ids']
        token_type_ids = tokenized['token_type_ids']
        attention_mask = tokenized['attention_mask']
        
#         print(str(self.sentences[i]) +' : ')
#         print(tokenized)
        
        return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 
                'attention_mask': attention_mask, 'label': self.labels[i]}
         
    def __len__(self): #data loader가 필요로 하여 필수적으로 있어야 하는 함수
        return len(self.sentences)

In [5]:
class collate_fn:
    def __init__(self, labels_dict):
        self.num_labels = len(labels_dict)
        
    def __call__(self, batch): #batch는 dataset.getitem의 return 값의 List. eg. [{}, {}. ...]
        #batch내 최대 문장 길이(토큰 개수)를 먼저 구해서 padding할 수 있도록 하기
        batchlen = [sample['input_ids'].size(1) for sample in batch] #tensor값을 반환하기 때문에 1번째 차원의 길이를 구함
        maxlen = max(batchlen)
        input_ids = []
        token_type_ids = []
        attention_mask = []
        #padding: [5, 6] [0, 0,  ...]을 concatenate 하는 방식으로 패딩
        for sample in batch:
            pad_len = maxlen - sample['input_ids'].size(1)
            pad = torch.zeros((1, pad_len), dtype=torch.int)            
            input_ids.append(torch.cat([sample['input_ids'], pad], dim=1))
            token_type_ids.append(torch.cat([sample['token_type_ids'], pad], dim=1))
            attention_mask.append(torch.cat([sample['attention_mask'], pad], dim=1))
        #batch 구성
        input_ids = torch.cat(input_ids, dim=0)
        token_type_ids = torch.cat(token_type_ids, dim=0)
        attention_mask = torch.cat(attention_mask, dim=0)
        
        #one-hot encoding
        #batch 내 라벨을 tensor로 변환
        tensor_label = torch.tensor([sample['label'] for sample in batch])
        
        return input_ids, token_type_ids, attention_mask, tensor_label

In [6]:
df = pd.read_csv('data/twitter_clean.csv', encoding="UTF-8")
print(len(df['y'].value_counts()))
df['y'].value_counts(sort = True).head(10)

30


N    235
😂     80
❤     53
😭     49
😍     32
💕     28
💜     26
🔥     25
👍     24
🥺     24
Name: y, dtype: int64

In [7]:
df['split'] = np.random.randn(df.shape[0], 1)
msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [8]:
train = sentencEmojiDataset('data/train.csv', tokenizer)
test = sentencEmojiDataset('data/test.csv', tokenizer)

train_collate_fn = collate_fn(train.labels_dict)
test_collate_fn = collate_fn(test.labels_dict)

train_collate_fn

In [9]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-3

In [10]:
train_dataloader = DataLoader(train, batch_size=batch_size, collate_fn=train_collate_fn, shuffle = True, drop_last = True)
test_dataloader = DataLoader(test, batch_size=batch_size, collate_fn=test_collate_fn, shuffle = False, drop_last = False)
train_dataloader

In [11]:
# class BERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size = 768,
#                  num_classes=2,
#                  dr_rate=None,
#                  params=None):
#         super(BERTClassifier, self).__init__()
#         self.bert = bert
#         # do not train bert parameters
#         for p in self.bert.parameters():
#             p.requires_grad = False
#         self.dr_rate = dr_rate
                 
#         self.classifier = nn.Linear(hidden_size , num_classes)
        
#         if dr_rate:
#             self.dropout = nn.Dropout(p=dr_rate)

#     def forward(self, input_ids, token_type_ids, attention_mask):
#         #eval: drop out 중지, batch norm 고정과 같이 evaluation으로 모델 변경
#         self.bert.eval()
#         #gradient 계산을 중지
#         with torch.no_grad():
#             x = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
# #         x = self.dropout(pooler)
#         return self.classifier(x)

In [12]:
class ELECTRAClassifier(nn.Module):
    def __init__(self,
                 electra,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        
        super(ELECTRAClassifier, self).__init__()
        
        self.electra = electra
        
        # do not train electra parameters
        for p in self.electra.parameters():
            p.requires_grad = False
        
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, input_ids, token_type_ids, attention_mask):
        #eval: drop out 중지, batch norm 고정과 같이 evaluation으로 모델 변경
        self.electra.eval()
        
        #gradient 계산을 중지
        with torch.no_grad():
            #ElectraModel은 pooled_output을 리턴하지 않는 것을 제외하고 BertModel과 유사합니다.
#            x = self.electra(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output

            x = self.electra(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :] 
            #.last_hidden_state[:, 0, :]: [batch , CLS 위치, depth]
            
            #Sentence Embedding으로 무엇을 넣을까? CLS, average, ... (Bert의 경우에는 Sentence BERT라는 게 제안되었다고 함)
                   
        x = self.dropout(x)
      
        return self.classifier(x)

In [13]:
label = list(set(list(df.iloc[:,1])))

In [14]:
# model = BERTClassifier(model,  dr_rate=0.5, num_classes = len(label))
model = ELECTRAClassifier(model,  dr_rate=0.5, num_classes = len(label))

In [15]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# optimizer = AdamW(model.parameters(), lr=learning_rate)

In [17]:
np.unique(train.labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [18]:
train.labels[:20]

[17, 21, 5, 13, 10, 17, 5, 28, 5, 5, 5, 17, 27, 28, 5, 5, 5, 22, 28, 5]

In [19]:
#Class Imbalance 문제 해결을 위한 weighted cross entropy 
class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train.labels), y = train.labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights) #([1.0000, 1.0000, 4.0000, 1.0000, 0.5714])

tensor([2.7417, 2.4370, 1.8278, 1.5667, 1.6872, 0.1313, 1.3708, 1.6872, 1.8278,
        1.3708, 1.5667, 1.4622, 1.0444, 1.1544, 2.1933, 1.2902, 1.2902, 0.4387,
        1.6872, 1.6872, 1.6872, 0.9536, 1.2902, 1.6872, 1.2185, 1.8278, 1.6872,
        1.3708, 0.6267, 0.7563])


In [20]:
train.labels_dict

{'key': range(0, 30),
 'value': ['💗',
  '😑',
  '🤤',
  '💙',
  '😘',
  'N',
  '😊',
  '🐹',
  '😁',
  '💚',
  '🥰',
  '😆',
  '💕',
  '🥺',
  '👉',
  '❣',
  '✨',
  '😂',
  '💖',
  '✌',
  '💯',
  '😍',
  '👍',
  '🤭',
  '💜',
  '😎',
  '🙏',
  '🔥',
  '❤',
  '😭']}

In [21]:
loss_fn = nn.CrossEntropyLoss(weight = class_weights, reduction = 'mean') 

In [22]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [23]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    loss_sum = 0
    model.train()
    for batch_id, (input_ids, token_type_ids, attention_mask, tensor_label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        out = model(input_ids, token_type_ids, attention_mask)
        loss = loss_fn(out, tensor_label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
#         scheduler.step()  # Update learning rate schedule
        batch_acc = calc_accuracy(out, tensor_label)
        train_acc += batch_acc
        loss_sum += loss.data.cpu().numpy()
        #f batch_id % log_interval == 0:
        print("epoch {} batch id {}/{} loss {} train acc {}".format(e+1, batch_id+1, len(train_dataloader), loss.data.cpu().numpy(), batch_acc))
    print("epoch {} train acc {} loss mean {}".format(e+1, train_acc / (batch_id+1), loss_sum / len(train_dataloader)))
    model.eval()
    with torch.no_grad():
        for batch_id, (input_ids, token_type_ids, attention_mask, tensor_label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
            out = model(input_ids, token_type_ids, attention_mask)
            test_acc += calc_accuracy(out, tensor_label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1/10 loss 3.572819232940674 train acc 0.03125
epoch 1 batch id 2/10 loss 3.4056732654571533 train acc 0.03125
epoch 1 batch id 3/10 loss 3.5389044284820557 train acc 0.0
epoch 1 batch id 4/10 loss 3.518869400024414 train acc 0.015625
epoch 1 batch id 5/10 loss 3.4964404106140137 train acc 0.015625
epoch 1 batch id 6/10 loss 3.394656181335449 train acc 0.046875
epoch 1 batch id 7/10 loss 3.6041438579559326 train acc 0.0
epoch 1 batch id 8/10 loss 3.5636298656463623 train acc 0.046875
epoch 1 batch id 9/10 loss 3.5690269470214844 train acc 0.015625
epoch 1 batch id 10/10 loss 3.528172254562378 train acc 0.03125
epoch 1 train acc 0.0234375 loss mean 3.5192335844039917


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:36<00:00,  7.27s/it]


epoch 1 test acc 0.02267156862745098
epoch 2 batch id 1/10 loss 3.5905935764312744 train acc 0.0
epoch 2 batch id 2/10 loss 3.590209722518921 train acc 0.03125
epoch 2 batch id 3/10 loss 3.442885160446167 train acc 0.03125
epoch 2 batch id 4/10 loss 3.4319005012512207 train acc 0.03125
epoch 2 batch id 5/10 loss 3.698812484741211 train acc 0.015625
epoch 2 batch id 6/10 loss 3.493004083633423 train acc 0.03125
epoch 2 batch id 7/10 loss 3.682891845703125 train acc 0.0
epoch 2 batch id 8/10 loss 3.550597906112671 train acc 0.03125
epoch 2 batch id 9/10 loss 3.4911088943481445 train acc 0.046875
epoch 2 batch id 10/10 loss 3.493788719177246 train acc 0.0
epoch 2 train acc 0.021875 loss mean 3.54657928943634


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.19s/it]


epoch 2 test acc 0.02267156862745098
epoch 3 batch id 1/10 loss 3.528674602508545 train acc 0.03125
epoch 3 batch id 2/10 loss 3.4830775260925293 train acc 0.015625
epoch 3 batch id 3/10 loss 3.5366852283477783 train acc 0.015625
epoch 3 batch id 4/10 loss 3.523273229598999 train acc 0.03125
epoch 3 batch id 5/10 loss 3.532040596008301 train acc 0.03125
epoch 3 batch id 6/10 loss 3.5267746448516846 train acc 0.0
